In [3]:
# imports
import pandas as pd
import numpy as np
import requests
import os
import json
import api_key

In [63]:
# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [28]:
API_KEY = api_key.foursquare_api['key']

In [8]:
# importing result from citybikes API with all stations in Toronto
df = pd.read_csv(r'C:\Users\rekha\Downloads\Bike_stations.csv')
df

,Station Name,Latitude,Longitude,Bikes Available
0,Queen St E / Woodward Ave,43.665269,-79.319796,18
1,Primrose Ave / Davenport Rd,43.671420,-79.445947,8
2,Queen St E / Rhodes Ave,43.666224,-79.317693,15
3,Bond St / Queen St E,43.653236,-79.376716,2
4,Church St / Alexander St,43.663722,-79.380288,17
...,...,...,...,...
754,Opposite 1103 Leslie St,43.720932,-79.349894,0
755,Widmer St / Adelaide St W,43.647548,-79.391552,6
756,800 Fleet St (North),43.636829,-79.408842,23
757,800 Fleet St (South),43.636710,-79.408805,25


In [12]:
 # Defining API endpoint
url = "https://api.foursquare.com/v3/places/search"

# headers
headers = {"Accept": "application/json",
           "Authorization": API_KEY}

In [22]:
 params = {
        'query': 'Indian Restaurant',
        'll': f"{df.loc[0,'Latitude']},{df.loc[0, 'Longitude']}",
        'radius': 1000,
        'limit': 3,      # Limit the number of results (adjust as needed)
    
        'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
    }
   
params    


{'query': 'Indian Restaurant',
 'll': '43.665269,-79.319796',
 'radius': 1000,
 'limit': 3,
 'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'}

In [23]:
 # Send the Foursquare API request
response = requests.get(url, params=params, headers=headers)

In [24]:
df.loc[0,'Latitude']
df.loc[0, 'Longitude']

-79.319796

In [25]:
response

<Response [200]>

In [29]:
# response.json()

In [32]:
# Creating empty list to store responses
response_list = []

# Iterating through each station in Bike_stations
for station in range(len(df)):
    latitude = df.loc[station,'Latitude']
    longitude =df.loc[station, 'Longitude']

    # Defining API endpoint
    url = "https://api.foursquare.com/v3/places/search"

    # Headers
    headers = {"Accept": "application/json",
                 "Authorization": API_KEY}

    # Setting up parameters for API request
    params = {
        'query': 'Indian Restaurant',
        'll': f"{latitude},{longitude}",
        'radius': 1000,
        'limit': 3,      # Limit the number of results (adjust as needed)
    
        'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
    }


    # Send the Foursquare API request
    response = requests.get(url, params=params, headers=headers)

    # Check if request was successful (status code 200)
    if response.status_code == 200:
        # Use the .json() method to parse the response content into JSON
        data_json = response.json()
        
        # Append the response to the list
        response_list.append(data_json)
    else:
        print(f"Request failed: {response.status_code}")



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [38]:
flattened_data = [
    {
    "Latitude": response["geocodes"]["main"].get("latitude"),
    "Longitude": response["geocodes"]["main"].get("longitude"),
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("formatted_address"),
    "telephone": response.get("tel"),
    "rating": response.get("rating"),
    "open_true": response["hours"].get("open_now")
    }
  for result in response_list
  for response in result.get("results", [])
]



Put your parsed results into a DataFrame

In [39]:
indian_restaurant = pd.DataFrame(flattened_data)

indian_restaurant

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,43.672665,-79.321041,Udupi Palace,810,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",(416) 405-8189,8.7,True
1,43.672278,-79.322523,Motimahal Restaurant Ltd,798,"1422 Gerrard St E (Coxwell), Toronto ON M4L 1Z6",(416) 461-3111,8.1,True
2,43.671871,-79.324283,Bombay Chowpatty,812,"1386 Gerrard St E, Toronto ON M4L 1Z2",(416) 405-8080,7.3,False
3,43.676566,-79.456089,Honorary Consulate of the Kingdom of Cambodia ...,995,"60 Caledonia Rd, Toronto ON M6E 4S4",(647) 533-9335,NaN,False
4,43.677961,-79.443916,Pizza e Pazzi,733,"1182 Saint Clair Ave W, Toronto ON M6E 1B4",(647) 352-7882,8.1,False
...,...,...,...,...,...,...,...,...
1814,43.638768,-79.416315,Maurya East Indian Cuisine - Liberty Village,661,"150 E Liberty St, Toronto ON M6K 3R5",(647) 347-7002,NaN,True
1815,43.638479,-79.417204,Brazen Head Irish Pub,697,"165 E. Liberty St (at Lynn Williams St.), Toro...",(416) 535-8787,6.2,True
1816,43.667943,-79.385626,Kothur Indian Cuisine,747,"649 Yonge St, Toronto ON M4Y 1Z9",(416) 926-0953,7.8,False
1817,43.661085,-79.385750,Colaba Junction,339,"801 Bay St, Toronto ON M5S 1Y9",(416) 646-8800,6.4,True


In [97]:
# creating a csv file for Bike_stations Dataframe to use it for further steps
file_path = r'C:\Users\rekha\Downloads\indian_restaurant.csv'

# save the dataframe into CSV file path
indian_restaurant.to_csv(file_path, index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [103]:
import api_key2
Yelp_API_KEY = api_key2.yelp_api['key']


In [75]:
# Defining API endpoint
url_yelp = "https://api.yelp.com/v3/businesses/search"

# headers
headers = {"Accept": "application/json",
           "Authorization": Yelp_API_KEY}

In [76]:
 params = {
           'latitude': latitude,
           'longitude': longitude,
           'term': 'Indian Restaurant',
           'radius': 1000,
             }  
    
params 

{'latitude': 43.665269,
 'longitude': -79.319796,
 'term': 'Indian Restaurant',
 'radius': 1000}

In [77]:
response = requests.get(url_yelp, params=params, headers=headers)

In [78]:
df.loc[0,'Latitude']
df.loc[0, 'Longitude']

-79.319796

In [80]:
# creating empty list to store responses
yelp_response_list = []

# maximum number of request to send
MAX_REQUEST = 490
# initialize a request count
request_count = 0


# iteraing through each station in Bike_stations 
for index, row in df.iterrows():
    
    #check if the maximum number of request has been reached
    if request_count >= MAX_REQUEST:
        break
        
    latitude = row['Latitude']
    longitude = row['Longitude']
    
    
     # Defining API endpoint
    url_yelp = "https://api.yelp.com/v3/businesses/search"

    # headers
    headers = {"Accept": "application/json",
                 "Authorization": f'Bearer {Yelp_API_KEY}',
                 }
    

    
    
    # setting up parameters for API request

    params = {
           'latitude': latitude,
           'longitude': longitude,
           'term': 'Indian Restaurant',
           'radius': 1000,
             }  
    
    # send the foresquare API request
    response_yelp = requests.get(url=url_yelp,params=params, headers=headers)
    
    
    # check if request was successfull (status code 200)
    if response_yelp.status_code == 200:
        #use the .json() method to parse the response content into json
        yelp_data_json = response_yelp.json()
        
        # Append the response to the list
        yelp_response_list.append(yelp_data_json)
        
        # Increment the request counter
        request_count += 1
    else:
        print(f"Request failed: {response_yelp.status_code}")
    

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc).
# My POI is 'indian Restaurants' in the above piece of code.

In [82]:
yelp_flattened_data = [
    {
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("display_address"),
    "telephone": response.get("display_phone"),
    "rating": response.get("rating"),
    "review_counts": response.get("review_count"),
    "closed": response.get("is_closed")
    }
  for result in yelp_response_list
  for response in result.get("businesses", [])
]

Put your parsed results into a DataFrame

In [83]:
yelp_indian_restaurant = pd.DataFrame(yelp_flattened_data)
yelp_indian_restaurant

,name,distance,address,telephone,rating,review_counts,closed
0,The Famous Indian Cuisine,809.806740,"[1410 Gerrard Street E, Toronto, ON M4L 1Z4, C...",+1 416-406-4511,4.0,66,False
1,Lahore Tikka House,787.473798,"[1365 Gerrard Street E, Toronto, ON M4L 1Z3, C...",+1 416-406-1668,3.5,546,False
2,Leela Indian Food Bar,829.086102,"[1386 Gerrard Street E, Toronto, ON M4L 1Z2, C...",+1 416-461-8088,4.0,20,False
3,Chandni Chowk Restaurant,816.858497,"[1426 Gerrard Street E, Toronto, ON M4L 1Z6, C...",+1 416-469-4466,4.0,32,False
4,Udupi Palace,839.396242,"[1460 Gerrard Street E, Toronto, ON M4L 2A3, C...",+1 416-405-8189,3.5,208,False
...,...,...,...,...,...,...,...
5014,Curryish Tavern,988.930837,"[783 Queen Street W, Toronto, ON M6J 1G1, Canada]",+1 416-392-7837,4.0,11,False
5015,My Roti Place,654.277866,"[901 King Street W, Toronto, ON M5V 3H5, Canada]",+1 416-306-8445,5.0,1,False
5016,Sizzler Kabab,697.263479,"[754 King Street W, Toronto, ON M5V 1N3, Canada]",,3.5,11,False
5017,Coco Rice Thai Cuisine,754.553356,"[669 King Street W, Toronto, ON M5V 1M9, Canada]",+1 416-504-9434,3.5,114,False


In [99]:
file_path = r'C:\Users\rekha\Downloads\yelp_indian_restaurant.csv'

# save the dataframe into CSV file path
yelp_indian_restaurant.to_csv(file_path, index=False)


In [101]:
indian_restaurant = pd.read_csv(r'C:\Users\rekha\Downloads\indian_restaurant.csv')


# Comparing Results

## We'll review the results from the Foursquare API, specifically the data related to Indian restaurants.

In [102]:
indian_restaurant

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,43.672665,-79.321041,Udupi Palace,810,"1460 Gerrard St E (Coxwell Ave), Toronto ON M4...",(416) 405-8189,8.7,True
1,43.672278,-79.322523,Motimahal Restaurant Ltd,798,"1422 Gerrard St E (Coxwell), Toronto ON M4L 1Z6",(416) 461-3111,8.1,True
2,43.671871,-79.324283,Bombay Chowpatty,812,"1386 Gerrard St E, Toronto ON M4L 1Z2",(416) 405-8080,7.3,False
3,43.676566,-79.456089,Honorary Consulate of the Kingdom of Cambodia ...,995,"60 Caledonia Rd, Toronto ON M6E 4S4",(647) 533-9335,NaN,False
4,43.677961,-79.443916,Pizza e Pazzi,733,"1182 Saint Clair Ave W, Toronto ON M6E 1B4",(647) 352-7882,8.1,False
...,...,...,...,...,...,...,...,...
1814,43.638768,-79.416315,Maurya East Indian Cuisine - Liberty Village,661,"150 E Liberty St, Toronto ON M6K 3R5",(647) 347-7002,NaN,True
1815,43.638479,-79.417204,Brazen Head Irish Pub,697,"165 E. Liberty St (at Lynn Williams St.), Toro...",(416) 535-8787,6.2,True
1816,43.667943,-79.385626,Kothur Indian Cuisine,747,"649 Yonge St, Toronto ON M4Y 1Z9",(416) 926-0953,7.8,False
1817,43.661085,-79.385750,Colaba Junction,339,"801 Bay St, Toronto ON M5S 1Y9",(416) 646-8800,6.4,True


In [86]:
# check any null values
indian_restaurant.isnull().sum()

Latitude       0
Longitude      0
name           0
distance       0
address        0
telephone    130
rating       878
open_true      0
dtype: int64

In [87]:
# checking duplicating values 
indian_restaurant.duplicated().sum()

17

In [88]:
indian_restaurant.shape

(1819, 8)

In [89]:
yelp_indian_restaurant.shape

(5019, 7)

In [90]:
# checking null values
yelp_indian_restaurant.isnull().sum()

name             0
distance         0
address          0
telephone        0
rating           0
review_counts    0
closed           0
dtype: int64

# DataFrame column containing lists, and lists are unhashable types, so Pandas cannot determine whether elements are duplicates.

# To handle this, we need to specify a subset of columns to check for duplicates, and we may need to convert the lists to tuples, as tuples are hashable

In [92]:
yelp_indian_restaurant['address'] = yelp_indian_restaurant['address'].apply(tuple)

# Now you can check for duplicates
duplicates = yelp_indian_restaurant.duplicated(subset=['address'])

# Count the number of duplicates
num_duplicates = duplicates.sum()

In [93]:
# checking duplicate values

yelp_indian_restaurant.duplicated().sum()

0

Which API provided you with more complete data? Provide an explanation. 

# Yelp API provided more complete data as mentioned in above steps, There are no null and duplicated value in yelp result, whereas in foursquare there are null results in ratings and telephone numbers,  Yelp provides a greater number of field attributes in its results compared to Foursquare 

Get the top 10 restaurants according to their rating

In [95]:
# indian_restaurants foursquare result 
# indian_restaurant.sort_values(by='rating',ascending=False).head(10)
indian_restaurant_result = indian_restaurant.sort_values(by='rating', ascending=False).head(10)[['name', 'distance', 'address', 'telephone', 'rating', 'open_true']]
indian_restaurant_result

,name,distance,address,telephone,rating,open_true
68,Pukka,505,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
1091,Pukka,913,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
379,Pukka,869,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
381,Pukka,879,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
223,Pukka,363,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
1279,Pukka,590,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
873,Pukka,854,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
787,Pukka,272,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
1287,Pukka,452,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False
804,Pukka,779,"778 St Clair Ave W (at Arlington Ave), Toronto...",(416) 342-1906,9.0,False


# This is including the duplicates as well. We will keep the duplicated results for now as looking at distance it is duplicating cause the restaurants might be showing for different bike stations.

In [96]:
# indian restaurant in yelp API
yelp_indian_restaurant.sort_values(by='rating', ascending=False).head(10)

,name,distance,address,telephone,rating,review_counts,closed
2055,Momo Ghar,700.635112,"(568 Parliament St, Toronto, ON M4X 1P6, Canada)",+1 416-920-8224,5.0,12,False
3114,Indraprastha,887.098281,"(5 Baldwin Street, Toronto, ON M5T 1L1, Canada)",+1 437-880-8349,5.0,2,False
4299,Poppadum,1268.706174,"(874 Yonge Street, Toronto, ON M4W 2J1, Canada)",+1 647-892-2907,5.0,6,False
4298,Indraprastha,965.466490,"(5 Baldwin Street, Toronto, ON M5T 1L1, Canada)",+1 437-880-8349,5.0,2,False
475,Indraprastha,880.064700,"(5 Baldwin Street, Toronto, ON M5T 1L1, Canada)",+1 437-880-8349,5.0,2,False
1320,Curry Kitchen + Falafel,867.019064,"(139 Dundas Street E, Toronto, ON M5B 2N6, Can...",+1 647-352-8779,5.0,1,False
3112,Globol Restaurants,328.636640,"(60 Carlton Street, Loblaws Maple Leaf Gardens...",,5.0,1,False
1317,Momo Ghar,921.645882,"(568 Parliament St, Toronto, ON M4X 1P6, Canada)",+1 416-920-8224,5.0,12,False
3116,Cucini Xpress,135.740467,"(372 Yonge Street, Toronto, ON M5G 2K9, Canada)",+1 416-827-1815,5.0,4,False
4176,Rikki Tikki,775.647675,"(209 Augusta Avenue, Toronto, ON M5T 2L4, Canada)",+1 416-792-3229,5.0,29,False


# Overall, Yelp returns more precise and abundant results than Foursquare.

# A limitation of Yelp is its lower API request quota compared to Foursquare. Consequently, extra caution is required in coding to avoid exhausting the allotted number of requests.